# IMPORT LIBRARY

In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
import time
import re

# START SCRAPING

## URL SCRAPING

In [117]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)

all_link = []
for i in range(1,11):
    driver.get(f'https://www.wongnai.com/businesses?q=%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%99%E0%B8%AD%E0%B8%B2%E0%B8%AB%E0%B8%B2%E0%B8%A3%E0%B8%AD%E0%B8%AD%E0%B8%A3%E0%B9%8C%E0%B9%81%E0%B8%81%E0%B8%99%E0%B8%B4%E0%B8%84&originalQ=%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%99%E0%B8%AD%E0%B8%B2%E0%B8%AB%E0%B8%B2%E0%B8%A3%E0%B8%AD%E0%B8%AD%E0%B8%A3%E0%B9%8C%E0%B9%81%E0%B8%81%E0%B8%99%E0%B8%B4%E0%B8%84&regions=9681&page.size=10&rerank=false&domain=1&page.number={i}')
    time.sleep(3)
    
    driver.execute_script("document.body.style.zoom='10%'")
    
    time.sleep(3)
    data = driver.page_source
    soup = bs(data)
    
    all_item = soup.find_all("a",{'class':'k0pvs2-0 sc-1bs98dy-0 hgwkbN ectdXQ'})
    

    for i in all_item:
        p = 'https://www.wongnai.com/'+i.get('href')
        all_link.append(p)
    
    

## CONTENT SCRAPING

In [147]:
name = []
price_range = []
cat = []
timeopen = []
telephone = []
review = []
place = []
rating = []

In [148]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)

for link in all_link:   
    driver.get(link)
    time.sleep(2)
    driver.execute_script("document.body.style.zoom='10%'")
    data = driver.page_source
    superman = bs(data)

# name
    for i in superman.find_all('span',{'class':'bd36 bd18-mWeb text-gray-700'}):
        name.append(i.get_text())
    
# pricerange
    if len(superman.find_all('span',{'class':'no92pk-0 vSGLf text-gray-500 rg16 rg14-mWeb'})) == 0:
        price_range.append('NoInfo')
    else:
        for i in superman.find_all('span',{'class':'no92pk-0 vSGLf text-gray-500 rg16 rg14-mWeb'}):
            i = i.get_text()
            if i == '฿':
                i = '<100'
                price_range.append(i)
            elif i == '฿฿':
                i = '101-250'
                price_range.append(i)
            elif i == '฿฿฿':
                i = '251-500'
                price_range.append(i)
            elif i == '฿฿฿฿':
                i = '501-1000'
                price_range.append(i)
            elif i == '฿฿฿฿฿':
                i = '>1000'
                price_range.append(i)
    
    
# style restaurant
    text = ''
    for i in superman.find_all('span',{'class':'sc-bdfBQB edNVkU rg16 rg14-mWeb font-highlight'}):
        i = i.get_text()
        text += f'{i},'
    text = text[:-1]
    cat.append(text)


# time
    try:
        for i in superman.find('table',{'class':'sc-1kh6w3g-8 gdNTro'}):
            i = i.find_all('td')[1].get_text()
            timeopen.append(i)
    except TypeError:
        timeopen.append('NoInfo')
    
# telephone
    i = superman.find_all('div',{'class':'rg16 rg14-mWeb font-highlight'})[1].get_text()
    i = re.sub("เบอร์โทร : ","",i)
    telephone.append(i)


# rating
    for i in superman.find_all('span',{'class':'rg16 rg14-mWeb font-highlight text-gray-500'}):
        i = i.get_text()
        i = i.replace(" ","")
        i = i.replace("(",",")
        i = i.replace(")","")
        review.append(i)
    
# map
    i = superman.find_all('div',{'class':'rg16 rg14-mWeb font-highlight'})[0].get_text()
    place.append(i)


# review
    i = superman.find_all('div',{'class':'qkywve-1 dAZpcF bd48 text-gray-700'})
    if len(i) == 0:
        rating.append("NoRating")
    else:
        i = i[0].get_text()
        rating.append(i)


In [155]:
final = pd.DataFrame({'Name': name,
                     'PriceRange' : price_range,
                     'Categories': cat,
                     'Time': timeopen,
                     'Telephone': telephone,
                     'Reviews':review,
                     'Place': place,
                     'Rating': rating})

In [157]:
final.to_csv('wongnaiscrap.csv',index = False,encoding='utf-8')

In [149]:
for i in [name,
price_range,
cat,
timeopen,
telephone,
review ,
place, 
rating]:
    print(len(i))

100
100
100
100
100
100
100
100


In [156]:
final

,Name,PriceRange,Categories,Time,Telephone,Reviews,Place,Rating
0,ORGANIC SUPPLY (ออร์แกนิค ซัพพลาย),101-250,"อาหารมังสวิรัติ,เบเกอรี/เค้ก",09:00 - 19:00,020707996,"155เรตติ้ง,116รีวิว","ถนนนาคนิวาส, ลาดพร้าว, กรุงเทพมหานคร, 10230, ป...",4.2
1,Salada Organic Kitchen (ซาลาด้า ออร์แกนิค คิทเ...,101-250,"อาหารคลีน/สลัด,อาหารมังสวิรัติ,อาหารไทย",09:00 - 18:30,080-221-9490,"26เรตติ้ง,20รีวิว",105/1 ถนนเทศบาลสงเคราะห์ กรุงเทพมหานคร (ตลาดบอ...,3.7
2,ชาบูบ้านผัก ออร์แกนิค,101-250,ชาบู/สุกี้ยากี้/หม้อไฟ,10:00 - 21:00,0842645941,"3เรตติ้ง,1รีวิว",44/56 ซอย หมู่บ้านธนบุรี ตำบล ศาลากลาง อำเภอบา...,5
3,Salada Organic Kitchen (ซาลาด้า ออร์แกนิค คิทเ...,101-250,"อาหารคลีน/สลัด,อาหารไทย",10:00 - 20:00,098-360-5155,"4เรตติ้ง,3รีวิว","บางนา, กรุงเทพมหานคร, 10260, ประเทศไทย กรุงเทพ...",3.5
4,สวงอยู่ที่วังหลัง ออร์แกนิคคาเฟ่ (Sawong at Wa...,101-250,"พิซซ่า,คาเฟ่,เบเกอรี/เค้ก",10:30 - 19:30,0970300142,"53เรตติ้ง,39รีวิว","156/12 ซอยอรุณอมรินทร์ 22, ศิริราช, กรุงเทพมหา...",3.7
...,...,...,...,...,...,...,...,...
95,กอบัวฟาร์ม โฮมเมด ออร์แกนิคฟู้ดส์ (กอบัวฟาร์ม ...,<100,"อาหารจานเดียว,เครื่องดื่ม/น้ำผลไม้",NoInfo,0624469514,"0เรตติ้ง,0รีวิว",17/12 หมู่บ้านการไฟฟ้าส่วนภูมิภาค2 แขวงทุ่งสอง...,NoRating
96,ตามใจคาเฟ่ (Taamjai Cafe),101-250,คาเฟ่,09:30 - 18:00,026905591,"7เรตติ้ง,7รีวิว","73 Soi Suwanmanee, กรุงเทพมหานคร",3.7
97,สวนผัก โอ้กะจู๋,251-500,อาหารคลีน/สลัด,09:00 - 20:00,0944470914,"8เรตติ้ง,6รีวิว","1011, ถ.เทพรัตน บางนา บางนา กรุงเทพมหานคร กรุง...",4.7
98,เชฟขนมถ้วย ออร์แกนิค (เชฟขนมถ้วย),<100,ของหวาน,07:00 - 16:00,0927454218,"3เรตติ้ง,0รีวิว",230/1ซอยปรีดีพนมค์42 กรุงเทพมหานคร (เข้าซอยมา1...,0
